In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math

## function

In [6]:
def preprocess(row):

    root="C:/Users/KABYADE/Desktop/Fraud_ML/"
    encoder_path = "fraud-detection/preprocessing/output/dict_all.obj"
    scalerfile = 'fraud-detection/preprocessing/output/scaler.sav'

    # loading scaler
    min_max_scaler = pickle.load(open(root + scalerfile, 'rb'))

    # loading encoder dictionary
    file = open(root + encoder_path,'rb')
    dict_encoder = pickle.load(file)
    file.close()

    # treating nan values
    row.drop(['V11','V14','V20','V5','V17','V18','V15','CLASS'], inplace=True)

    # updating types
    row['V10'] = float(row['V10'])
    row['V9'] = float(row['V9'])
    row['V24'] = float(row['V24'])

    row['V23'] = pd.to_datetime(row['V23'],format="%d/%m/%Y")
    row['V8'] = pd.to_datetime(row['V8'],format="%d/%m/%Y %H:%M:%S")

    row['V6'] = pd.to_datetime(row['V6'],format="%d/%m/%Y")
    row['V7'] = pd.to_datetime(row['V7'],format="%d/%m/%Y")

    # preprocess dates
    date_cols = ['V6','V7','V23','V8']
    years = {}
    date_rows = {}

    for l in date_cols:
        date_rows[l+'_month'] = row[l].month
        date_rows[l+'_day'] = row[l].day
        years[l+'_year'] = row[l].year
        row.pop(l)

    cos_cols = list(date_rows.keys())
    sin_cols = list(date_rows.keys())


    for i in range(len(cos_cols)):
        cos_cols[i] = 'COS(' + cos_cols[i] + ')'
        sin_cols[i] = 'SIN(' + sin_cols[i] + ')'

    # Normalize date cols
    date_row_scaled = min_max_scaler.transform(np.array(list(date_rows.values())).reshape(-1,8))
    date_row_scaled = date_row_scaled.reshape(8)

    cos_row_scaled = dict(zip(cos_cols, date_row_scaled))
    sin_row_scaled = dict(zip(sin_cols, date_row_scaled))

    date_row_cos = pd.Series(cos_row_scaled)
    date_row_sin = pd.Series(sin_row_scaled)
    date_row_cos = date_row_cos.apply(lambda x: math.cos(x))
    date_row_sin = date_row_sin.apply(lambda x: math.sin(x))

    year_row = pd.Series(years)

    new_row = pd.concat([row, date_row_cos, date_row_sin, year_row], axis=0)

    # encoding
    for col in dict_encoder.keys():
        if(new_row[col] in dict_encoder[col].keys()):
            new_row.replace(dict_encoder[col], inplace=True)
        else:
            new_row[col] = -1

    return new_row

In [7]:
path = 'C:/Users/KABYADE/Desktop/Fraud_ML/dataset/final.xlsx'
df = pd.read_excel(path, dtype = str)

In [8]:
rt="C:/Users/KABYADE/Desktop/Fraud_ML/"
encoder_path = rt+"fraud-detection/preprocessing/output/dict_all.obj"
scalerfile = rt+'fraud-detection/preprocessing/output/scaler.sav'

In [9]:
row = df.iloc[1]
row['V10'] = 643212
row['V9'] = 239806.0
row['V24'] = 777541

file = open( rt + "fraud-detection/preprocessing/output/row.obj","wb")
pickle.dump(row, file)
file.close()

In [10]:
row = pickle.load(open(rt + "fraud-detection/preprocessing/output/row.obj", "rb"))

In [11]:
type(row)

pandas.core.series.Series

In [ ]:
row

In [ ]:
new_row = preprocess(row)
new_row

In [14]:
new_file = open( rt + "fraud-detection/preprocessing/output/new_row.obj","wb")
pickle.dump(new_row, new_file)
new_file.close()

In [70]:
from pypmml import Model

filename = "fraud-detection/models/svm.pmml"
SVM = Model.load(root + filename)

In [72]:
res = SVM.predict(new_row)

In [77]:
res

predicted_isFraud    1.0
probability          1.0
probability_0        0.0
probability_1        1.0
Name: 0, dtype: float64